<a href="https://colab.research.google.com/github/Muntasir2179/vector-database-learning/blob/main/VD_Pinecone_symentic_search_%2B_metadata_filterting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone - symentic search with metadata filtering

* News articles.
* symbol, publishedDate, title, site

In [1]:
!pip install pinecone-client sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=12e23ef8a0ec55e9e600f67159478d1098e334a0ca8f050bc4d7fe33ed0c98f6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import time
import pandas as pd
import numpy as np

from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

In [3]:
pc = Pinecone(api_key="af8ce84a-ab95-401c-ba72-b5249afdb37a")

In [4]:
pc.list_indexes()

{'indexes': []}

## Prepare Data - News Articles

In [5]:
df = pd.read_csv('Articles.csv', encoding = "cp1252")
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [6]:
df['NewsType'].value_counts()

sports      1408
business    1284
Name: NewsType, dtype: int64

In [7]:
# removing null values bases on date
df['Date'] = df['Date'].fillna('')
len(df)

2692

## Create Embeddings and Vectors

* We will be using `average_word_embeddings_komninos` sentence-transformer to generate embeddings.

* The embedding dimension will be 300.

In [8]:
model = SentenceTransformer(model_name_or_path='average_word_embeddings_komninos')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/267M [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

In [9]:
model

SentenceTransformer(
  (0): WordEmbeddings(
    (emb_layer): Embedding(222305, 300)
  )
  (1): Pooling({'word_embedding_dimension': 300, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
print("Encoding articles....")
encoded_articles = model.encode(df['Article'].tolist(), show_progress_bar=True)

Encoding articles....


Batches:   0%|          | 0/85 [00:00<?, ?it/s]

In [11]:
df['article-encoded'] = encoded_articles.tolist()

In [12]:
df.head()

,Article,Date,Heading,NewsType,article-encoded
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business,"[0.23253199458122253, 0.024697260931134224, -0..."
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business,"[0.10716943442821503, -0.012101680040359497, -..."
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business,"[-0.04096980020403862, 0.06097852811217308, 0...."
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business,"[0.06540685892105103, 0.0188664011657238, 0.01..."
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business,"[0.12384109199047089, 0.02100343629717827, 0.0..."


## Setting up Pinecone

* Manually create am index named `news-articles` with embedding dimension of `300`

In [15]:
pc.list_indexes()

{'indexes': [{'dimension': 300,
              'host': 'news-articles-reg6nin.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'news-articles',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [16]:
index = pc.Index(name="news-articles")

## Upsert vector data

In [20]:
df.index[0]

0

In [25]:
df['article-encoded'].iloc[:10]

0    [0.23253199458122253, 0.024697260931134224, -0...
1    [0.10716943442821503, -0.012101680040359497, -...
2    [-0.04096980020403862, 0.06097852811217308, 0....
3    [0.06540685892105103, 0.0188664011657238, 0.01...
4    [0.12384109199047089, 0.02100343629717827, 0.0...
5    [0.11176614463329315, 0.04939669743180275, -0....
6    [0.08501481264829636, -0.012918072752654552, 0...
7    [0.09655191004276276, 0.009973482228815556, -0...
8    [0.07618804275989532, 0.07852353155612946, -0....
9    [0.1508762538433075, 0.01594051904976368, 0.03...
Name: article-encoded, dtype: object

In [26]:
len(df)

2692

In [33]:
from tqdm import tqdm

for i in tqdm(range(0, len(df)), desc="Upserting data:"):
  id = str(df.index[i])
  vector = df['article-encoded'].iloc[i]
  metadata = {
      'date': df.Date.iloc[i],
      'heading': df.Heading.iloc[i],
      'news-type': df.NewsType.iloc[i]
  }
  index.upsert(vectors=[(id, vector, metadata)], show_progress=False)

Upserting data:: 100%|██████████| 2692/2692 [01:59<00:00, 22.55it/s]


In [34]:
index.describe_index_stats()

{'dimension': 300,
 'index_fullness': 0.02692,
 'namespaces': {'': {'vector_count': 2692}},
 'total_vector_count': 2692}

## Running Query on the data

In [38]:
phrase = 'London: World oil prices rebounded Friday after the International Energy Agency declared there were signs "the tide will turn" following recent multi-year lows.'
encoded_phrase = model.encode(phrase).tolist()
encoded_phrase

[0.12006988376379013,
 0.06613529473543167,
 -0.01570199802517891,
 0.05036306008696556,
 -0.24763505160808563,
 -0.04709300026297569,
 -0.11500417441129684,
 -0.09522328525781631,
 -0.34210294485092163,
 0.13683545589447021,
 -0.17531953752040863,
 -0.08191581070423126,
 0.1266670674085617,
 -0.2614979147911072,
 -0.2089722901582718,
 0.06764000654220581,
 -0.1598949432373047,
 -0.13605158030986786,
 -0.07085258513689041,
 -0.19735883176326752,
 0.037193529307842255,
 -0.12009282410144806,
 0.17304947972297668,
 0.2541889548301697,
 0.05519717186689377,
 -0.1363675743341446,
 0.10824082791805267,
 0.18911023437976837,
 -0.10776358842849731,
 0.05082623288035393,
 0.10235434770584106,
 0.0038943500258028507,
 -0.004683000035583973,
 -0.11557334661483765,
 -0.07444258779287338,
 -0.1769777536392212,
 -0.007165239192545414,
 -0.00805729441344738,
 -0.03250394016504288,
 -0.17367352545261383,
 -0.03824494406580925,
 -0.13017717003822327,
 -0.037060946226119995,
 -0.027301179245114326,
 0.

In [47]:
result = index.query(
    vector=encoded_phrase,
    top_k=4,
    include_metadata=True,
    filter={
        'news-type': {"$eq": "business"}
    }
)

In [48]:
result

{'matches': [{'id': '78',
              'metadata': {'date': '3/13/2015',
                           'heading': 'oil market drops on iea price warning',
                           'news-type': 'business'},
              'score': 0.925098121,
              'values': []},
             {'id': '12',
              'metadata': {'date': '1/16/2015',
                           'heading': 'oil market rebounds on iea repor',
                           'news-type': 'business'},
              'score': 0.921157181,
              'values': []},
             {'id': '393',
              'metadata': {'date': '12/21/2015',
                           'heading': 'Oil extends slide on US production '
                                      'export worri',
                           'news-type': 'business'},
              'score': 0.92067492,
              'values': []},
             {'id': '335',
              'metadata': {'date': '10/27/2015',
                           'heading': 'oil prices retreat in asi

In [49]:
# getting all the ids that we got from the result of our query
result_ids = [int(result.matches[i].id) for i in range(0, len(result.matches))]
result_ids

[78, 12, 393, 335]

In [50]:
df.loc[result_ids]

,Article,Date,Heading,NewsType,article-encoded
78,London: World oil prices sank Friday after the...,3/13/2015,oil market drops on iea price warning,business,"[0.14035895466804504, 0.025874778628349304, 0...."
12,London: World oil prices rebounded Friday afte...,1/16/2015,oil market rebounds on iea repor,business,"[0.15884417295455933, 0.003660765243694186, 0...."
393,Singapore: Oil prices resumed their slide in A...,12/21/2015,Oil extends slide on US production export worri,business,"[0.09949826449155807, 0.0356883630156517, 0.00..."
335,Singapore: Oil prices extended losses in Asia ...,10/27/2015,oil prices retreat in asi,business,"[0.13742856681346893, 0.012257379479706287, -0..."


## Cleaning up

In [51]:
pc.delete_index('news-articles')
pc.list_indexes()

{'indexes': []}